# Privacera Shield protecting LangChain Application
This notebook shows how to use Privacera Shield Library with a LangChain application. To run this notebook you will need the following,


1.  Sign up for a free account at [Privacera AI Governance (PAIG)](https://privacera.ai). This is simple, all you need is your email address.
2.  Your OpenAI API Key. This will allow you to create your first OpenAI application governed by Privacera AI Governance.

# Step 1 - Install the Python packages
This step will take about a minute to complete.



In [ ]:
!pip install -qqq \
  langchain-community==0.2.0 \
  langchain-openai==0.1.7 \
  langchain==0.2.0 \
  privacera_shield==1.1.5

# Step 2 - Upload Privacera AI application configuration file into Collab
1. Log into your account in PAIG.
1. To create a new application, go to Application > AI Application and click the CREATE APPLICATION button on the right top. This will open a dialog box where you can enter the details of the application.
1. Navigate to Application -> AI Applications and select the application you want to download the configuration file for. Click on the DOWNLOAD APP CONFIG button from the right top to download the configuration file.
1. You will then upload the configuration file into the Collab notebook by following this step. After you run this step, click on Choose Files button and upload the AI application configuration file.

In [ ]:
from google.colab import files
uploaded = files.upload()
files = uploaded.keys()
if len(files) > 1:
  print("Upload only the application config json file")
else:
  app_config_file_content = uploaded[list(files)[0]].decode('UTF-8')

# Step 3 - Configure your OpenAI API Key into the environment.
Enter your OpenAI API key in the pop-up. This key is stored in the environment of the notebook and not uploaded to PAIG.

Hit enter after you have pasted the key.

In [ ]:
import os
from getpass import getpass

if os.environ.get("OPENAI_API_KEY") is None:
    openai_api_key = getpass("🔑 Enter your OpenAI API key and hit Enter:")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Step 4 - Import all the required packages

In [ ]:
import os
from privacera_shield import client as privacera_shield_client
from privacera_shield.exception import AccessControlException
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate

# utility function to wrap the output
def wrap_text(text, width=80):
    words = text.split()
    character_count = 0
    for word in words:
        if character_count + len(word) + 1 > width:  # Check if adding the word would exceed the width
            print("\n", end="")  # Start a new line
            character_count = 0  # Reset the character count for the new line
        print(word, end=" ")  # Print the word followed by a space
        character_count += len(word) + 1  # Update the character count

# Step 5 - Initialize the Privacera Shield library
This one line of code will initialize the Privacera Shield library so that it is ready to protect your LangChain application.

Tip: If you get an error in this step, it could be because you have already run this step once. You can ignore the error and continue. You can also try restarting the Kernel and start from the beginning with the proper JSON file and OpenAI API key.

In [ ]:
# Initialize Privacera Shield
privacera_shield_client.setup(frameworks=["langchain"], application_config=app_config_file_content)

# Step 6 - Initialize OpenAI based LLM object in LangChain
In this step, we are creating an LLM object for the OpenAI LLM and a prompt template.

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")  # (1)

llm = OpenAI(openai_api_key=api_key)
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

# Step 7 - Run the LLMChain with your question
In this step, we are going to run the LLMChain with the prompt asked by a user named 'testuser'. Note how we are passing the username by creating a Privacera Shield context object. Privacera Shield will intercept the prompt and the response coming from LLM for the testuser and run policies. The PAIG service scans both the prompt and response text and runs security policies. If the policies decide that the access is denied then an AccessControlException is thrown.

In [ ]:
# Let's assume the user is "testuser"
user = "testuser"
prompt_text = "Who was the first President of USA and where did they live?"
print(f"Prompt: {prompt_text}")
print()

llm_chain = prompt | llm
try:
    with privacera_shield_client.create_shield_context(username=user):
        response = llm_chain.invoke(prompt_text)
        print(f"LLM Response:")
        wrap_text(response)
except AccessControlException as e:
    # If access is denied, then this exception will be thrown. You can handle it accordingly.
    print(f"AccessControlException: {e}")

# Step 8 - Review the access audits in PAIG portal under Security menu option
Now you can log in to the PAIG portal and check under Security -> Access Audits section. You will see the audit record for the above run of your LangChain application.

You can click on the eye icon and see the details of the prompts send by the application to the LLM and the responses coming from the LLM.

The default policy in PAIG for the application monitors the flow and tags the contents of the prompt and response.

# Step 9 - Modify the policy of your AI application
Log into the PAIG portal and navigate to your application under Application section. Click on the Permissions tab and enable the 'CCPA Personal Information Redaction' policy by clicking on the toggle in the status column. This will start redacting the sensitive data in the Prompt and Reply.

Wait for few seconds and re-run Step 7 above. The reply will have the Location and Person name redacted, that is, replaced by <LOCATION> and <PERSON>. You can re-run Step 7 few times.

Then check the access audits and you will see the original response from the LLM and how Privacera AI redacted the sensitive information out of it.

You can modify the policy by clicking on the pencil icon and change the Prompt to Allow while keeping Reply to Redact.

Wait for few seconds and re-run Step 7. In the access audits, you will see that the prompt is unchanged while the response is redacted.

You can change the question in Step 7 and also try changing the policy to see the effect.

# Next steps
Continue your journey with PAIG by following these sections in the documentation link in the PAIG portal,
- Integrating with Langchain application
- Integrating with Python application
- Try the Privacera SecureChat
- PAIG User Guide